In [ ]:
fca_reference_head = sc.read_h5ad(referencepath.joinpath("v2_fca_biohub_head_10x_raw.h5ad"))
fca_reference_antenna = sc.read_h5ad(referencepath.joinpath("v2_fca_biohub_antenna_10x_raw.h5ad"))

fca_reference_head.layers["counts"] = fca_reference_head.X.copy()
fca_reference_antenna.layers["counts"] = fca_reference_antenna.X.copy()

fca_reference_head.obs['tissue'] = "fca_head"
fca_reference_head.obs['tissue_batch'] = fca_reference_head.obs['batch'].astype(str)+"_"+fca_reference_head.obs['tissue'].astype(str)
fca_reference_antenna.obs['tissue'] = "fca_antenna"
fca_reference_antenna.obs['tissue_batch'] = fca_reference_antenna.obs['batch'].astype(str)+"_"+fca_reference_antenna.obs['tissue'].astype(str)

print(fca_reference_head.var.index.is_unique)
print(fca_reference_antenna.var.index.is_unique)


fca_reference_combined = fca_reference_head.concatenate(fca_reference_antenna,join='outer', batch_key="concatbatch")
fca_reference_combined.layers["counts"] = fca_reference_combined.X.copy()
fca_reference_combined


In [ ]:
sca.models.SCVI.setup_anndata(fca_reference_combined, batch_key="tissue_batch", categorical_covariate_keys=["age","fly_genetics","tissue","tissue_batch","dissection_lab","sex"])

In [ ]:
sc.pp.normalize_total(fca_reference_combined)
sc.pp.log1p(fca_reference_combined)
fca_reference_combined.raw = fca_reference_combined
combined_vae = sca.models.SCVI(
    fca_reference_combined,
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=True,
    use_layer_norm="both",
    use_batch_norm="none"
)

In [ ]:
combined_vae.train(max_epochs=2, check_val_every_n_epoch = 1)